In [1]:
using PyPlot

INFO: Loading help data...


In [47]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

(fig, ax) = plt.subplots(8,8)
x=[1:1:100]
y=[rand(100)]
li=[hi[:plot](x, y)[1] for hi in ax]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:10
    
    #[fig[:canvas][:restore_region](hi) for hi in backgrounds]
    [hi[:set_ydata](y2[i,:]) for hi in li]
    [fig[:canvas][:blit](hi[:bbox]) for hi in ax]
    #fig[:canvas][:draw]()
end

elapsed time: 1.072894188 seconds (471040 bytes allocated)


In [1]:
using ExtractSpikes

In [20]:
function main()
    s=SpikeDetection()
    c=Cluster()
    be=ones(Int64,2000)
    #be[1000:1000:end]=10*be[1000:1000:end]
    prepareDetection(s,be)
    @time detectSpikes(s,c,be)
end

main (generic function with 1 method)

In [21]:
main()

elapsed time: 5.6511e-5 seconds (368 bytes allocated)


([0],[0])